## I - Objet :

- Produire une dataframe exploitable pour la datviz et pour l'entraînement. 

## II - Pré-requis :

- Disposer du dataset DSD 100 complet sur son disque.
- Retirer du DSD 100 les 2 éléments suivants:
    - le fichier excel intitulé dsd100.
    - le fichier text intitulé dsd.
- Corriger la coquille dans le dsd100 excel / ligne 62 : Patrick Talbot - "Set me Free" au lieu de "Set Free me".
- Avoir renseigné ci-après le path du fichier DSD propre à sa machine:

In [1]:
path = 'D:/DATASCIENTEST/00-PROJET_FIL_ROUGE-00/dataset/DSD100'

## III - Trame:

- On crée une liste contenant l'ensemble des chemins vers les fichiers musicaux.
- On crée une liste contenant le triptique Id - Artiste - Morceau.
- On utilise le fichier excel dsd100 pour obtenir le style musical associé à chaque titre.
- On crée la df_brute : les informations relatives au chemin réseau des fichiers musicaux.
- On crée alors les dataframes:
    - df_mixture : les seuls canaux mixtures.
    - df_vocal : les seuls canaux vocaux.
    - df_drums : les seuls canaux drums.
    - df_bass : les seuls canaux bass.
    - df_other : les seuls canaux others.
    - df_multi : tous les canaux sur une même ligne.



 !!! DECOMMENTER LES LIGNES DE SAUVEGARDE, A LA FIN DE CE NOTEBOOK, POUR GENERER LES FICHIERS UTILES POUR LA SUITE !!!


## IV - Code :

### IV.i - Bibliothèques requises

In [2]:
import os
import re
import subprocess
import pandas as pd
import numpy as np
import glob
import pathlib
import librosa
import librosa.display
import swifter
from IPython.display import Audio, display

### IV.ii - Fonctions dédiées

In [3]:
############################################################################################################
# On crée une fonction qui attribue l'Id - Artiste - Morceau' en fonction du path:
def nomme(path):
  
    for titre in titres:
        if titre in path:                 # On parcours le path avec le contenu de la liste des 'id - Artistes - Morceau'
            result = titre
    return result

############################################################################################################
# On crée une fonction qui permet de passer un texte dans une liste et en restitue un texte sans les 2 premiers éléments:
def filtre_num(x):
    r = re.compile(r"[A-Za-z0-9\-]+")
    
    liste = r.findall(x)
    return ' '.join(liste[2:])

############################################################################################################
# On crée une fonction qui intègre les élements d'un texte dans une liste et les restitue de façon 
#à avoir un espace entre chaque élément:
def syntax(x):
    r = re.compile(r"[A-Za-z0-9\-]+")
    
    liste = r.findall(x)
    return ' '.join(liste)

############################################################################################################
# On crée une fonction qui permettra d'identifier à quel canal correspond le path:
def canal(path):
    bout = path.split('wav')[0][-9:]              # On récupère un nombre de caractères suffisants en bout de chaine. 
    
    BOUT = []                                     # une liste intermédiaire de stockage de ces caractères.
    FINAL=[]                                      # une liste intermédiaire de stockage des caractères triés.
    
    for i in bout:                                # On stocke les caractères dans la liste et on en inverse l'ordre.
        BOUT.append(i)
    BOUT.reverse()
    BOUT.remove('.')
    
    for i in BOUT:                                # On parcours la liste jusqu'à rencontrer un séparateur quelconque.
        if ord(i) in [a for a in range(97,123)]:  # on convertit en caractère ascii (les lettres minuscules).
            FINAL.append(i)
        if ord(i) not in [a for a in range(97,123)]:
            break
          
    FINAL.reverse()    
    return ''.join(FINAL)

    


### IV.iii - Développements

- On crée une liste contenant les chemins vers les fichiers musicaux du DSD100.

In [4]:
liste = glob.glob(path+'/**/*.wav', recursive=True)

- On crée une liste contenant les titres des morceaux.

In [5]:
titres =[]

for root,dirs,files in os.walk(path): 
    for titre in dirs:
        titres.append(titre)
titres.remove('Mixtures') 
titres.remove('Sources')        
titres.remove('Dev') 
titres.remove('Test') 

- On utilise le fichier excel dsd100 afin de créer le dictionnaire permettant de faire la correspondance entre morceaux et styles.

In [6]:
data = pd.read_excel('dsd100.xlsx')                         # On charge les données dans un fichier excel
data.Name = data.Name.apply(lambda x: syntax(x))            # On reprend la syntaxe de la colonne Name

dictus = {}                                                 # Dictionnaire permettant d'associer un style à un artiste/morceau

for name,style in zip (data.Name.values,data.Style.values): 
    dictus[name] = style
    dictus[' '+name] = style
    dictus[' '+name+' '] =style
    dictus[name+' '] =style
    


- On bâtit la première dataframe de données brutes à 44100 Hz.


In [7]:
df_brute = pd.DataFrame(liste,columns =['Path'])                                            # le path
df_brute['signal_temporel']=df_brute['Path'].swifter.apply(lambda x: librosa.load (x, sr=None)[0])  # le signal

df_brute['Id_Artiste_Morceau'] = df_brute.Path.apply(lambda x: nomme(x))                    # le duo Artiste-Morceau
df_brute['Artiste_Morceau'] = df_brute['Id_Artiste_Morceau'].apply(lambda x: filtre_num(x)) # éviction du numéro
df_brute['Id'] = df_brute['Id_Artiste_Morceau'].apply(lambda x: x.split('-')[0] )           # le numéro
df_brute['Style'] = df_brute['Artiste_Morceau'].apply(lambda x: dictus[x])                  # le style
df_brute['canal']=df_brute['Path'].apply(lambda x: canal(x))                                # le canal

df_brute = df_brute.drop(['Id_Artiste_Morceau'],axis = 1)                                   # Suppression variable inutile

Pandas Apply:   0%|          | 0/500 [00:00<?, ?it/s]

- On bâtit les dataframes pour la dataviz.

In [8]:
# les dictionnaires de renommage de la colonne signal temporel:
rename_bass = {'signal_temporel':'bass'}
rename_drums = {'signal_temporel':'drums'}
rename_mixture = {'signal_temporel':'mixture'}
rename_other = {'signal_temporel':'other'}
rename_vocals = {'signal_temporel':'vocals'}

# la création des df contenant chacune 1 seul type de canal, en réordonnant:
df_bass = df_brute[df_brute['canal']=='bass'].drop(['Path','canal'],axis = 1).reset_index().drop(['index'],axis = 1).rename(rename_bass,axis = 1)[['Id','Artiste_Morceau','Style','bass']]
df_drums = df_brute[df_brute['canal']=='drums'].drop(['Path','canal'],axis = 1).reset_index().drop(['index'],axis = 1).rename(rename_drums,axis = 1)[['Id','Artiste_Morceau','Style','drums']]
df_mixture = df_brute[df_brute['canal']=='mixture'].drop(['Path','canal'],axis = 1).reset_index().drop(['index'],axis = 1).rename(rename_mixture,axis = 1)[['Id','Artiste_Morceau','Style','mixture']]
df_other = df_brute[df_brute['canal']=='other'].drop(['Path','canal'],axis = 1).reset_index().drop(['index'],axis = 1).rename(rename_other,axis = 1)[['Id','Artiste_Morceau','Style','other']]
df_vocals = df_brute[df_brute['canal']=='vocals'].drop(['Path','canal'],axis = 1).reset_index().drop(['index'],axis = 1).rename(rename_vocals,axis = 1)[['Id','Artiste_Morceau','Style','vocals']]

# la création de la dataframe contenant pour chaque Artiste / Morceau, l'ensemble des canaux:
df_multi = pd.concat([df_bass.Id,df_bass.Artiste_Morceau,df_bass.Style,df_bass.bass,df_drums.drums,df_mixture.mixture,df_other.other,df_vocals.vocals],axis = 1)

## V - RESULTATS :

In [9]:
df_multi

,Id,Artiste_Morceau,Style,bass,drums,mixture,other,vocals
0,051,AM Contra - Heart Peripheral,Electronic Dance Pop,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, -1.5258789e-05, 1.5258789e-05, 0.0, -1.5...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, -1.5258789e-05, 1.5258789e-05, 0.0, -1.5..."
1,052,ANiMAL - Easy Tiger,UK Hip-Hop,"[0.0, 0.0, 0.0, 0.0, 0.0, 3.0517578e-05, -1.52...","[0.22010803, 0.22981262, 0.25274658, 0.2085571...","[0.22012329, 0.2297821, 0.25274658, 0.20861816...","[0.0, -1.5258789e-05, 0.0, 1.5258789e-05, -3.0...","[0.0, 0.0, -1.5258789e-05, 1.5258789e-05, -3.0..."
2,053,Actions - Devil s Words,Power Pop,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,054,Actions - South Of The Water,Power Pop,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,055,Angels In Amplifiers - I m Alright,Smooth Acoustic Rock,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.525...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.5258789e...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.5258789e...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.525...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.525..."
...,...,...,...,...,...,...,...,...
95,046,Triviul - Dorothy,Leftfield Pop/Electronica,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
96,047,Voelund - Comfort Lives In Belief,Laid-back Blues-Rock,"[0.00048828125, 0.00039672852, 0.00036621094, ...","[4.5776367e-05, -1.5258789e-05, -3.0517578e-05...","[0.0027008057, 0.0027313232, 0.0015869141, 0.0...","[0.002166748, 0.0023498535, 0.0012512207, 0.00...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
97,048,We Fell From The Sky - Not You,High-Energy Heavy Rock,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
98,049,Young Griffo - Facade,Anthemic Heavy Rock,"[-3.0517578e-05, -3.0517578e-05, -3.0517578e-0...","[-0.00033569336, -0.00024414062, -0.0002136230...","[-0.00044250488, -0.00038146973, -0.0003356933...","[-4.5776367e-05, -6.1035156e-05, -4.5776367e-0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [10]:
df_bass.head()

,Id,Artiste_Morceau,Style,bass
0,051,AM Contra - Heart Peripheral,Electronic Dance Pop,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,052,ANiMAL - Easy Tiger,UK Hip-Hop,"[0.0, 0.0, 0.0, 0.0, 0.0, 3.0517578e-05, -1.52..."
2,053,Actions - Devil s Words,Power Pop,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,054,Actions - South Of The Water,Power Pop,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,055,Angels In Amplifiers - I m Alright,Smooth Acoustic Rock,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.525..."


In [11]:
df_drums.head()

,Id,Artiste_Morceau,Style,drums
0,051,AM Contra - Heart Peripheral,Electronic Dance Pop,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,052,ANiMAL - Easy Tiger,UK Hip-Hop,"[0.22010803, 0.22981262, 0.25274658, 0.2085571..."
2,053,Actions - Devil s Words,Power Pop,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,054,Actions - South Of The Water,Power Pop,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,055,Angels In Amplifiers - I m Alright,Smooth Acoustic Rock,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.5258789e..."


In [12]:
df_mixture.head()

,Id,Artiste_Morceau,Style,mixture
0,051,AM Contra - Heart Peripheral,Electronic Dance Pop,"[0.0, -1.5258789e-05, 1.5258789e-05, 0.0, -1.5..."
1,052,ANiMAL - Easy Tiger,UK Hip-Hop,"[0.22012329, 0.2297821, 0.25274658, 0.20861816..."
2,053,Actions - Devil s Words,Power Pop,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,054,Actions - South Of The Water,Power Pop,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,055,Angels In Amplifiers - I m Alright,Smooth Acoustic Rock,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.5258789e..."


In [13]:
df_other.head()

,Id,Artiste_Morceau,Style,other
0,051,AM Contra - Heart Peripheral,Electronic Dance Pop,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,052,ANiMAL - Easy Tiger,UK Hip-Hop,"[0.0, -1.5258789e-05, 0.0, 1.5258789e-05, -3.0..."
2,053,Actions - Devil s Words,Power Pop,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,054,Actions - South Of The Water,Power Pop,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,055,Angels In Amplifiers - I m Alright,Smooth Acoustic Rock,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.525..."


In [14]:
df_vocals.head()

,Id,Artiste_Morceau,Style,vocals
0,051,AM Contra - Heart Peripheral,Electronic Dance Pop,"[0.0, -1.5258789e-05, 1.5258789e-05, 0.0, -1.5..."
1,052,ANiMAL - Easy Tiger,UK Hip-Hop,"[0.0, 0.0, -1.5258789e-05, 1.5258789e-05, -3.0..."
2,053,Actions - Devil s Words,Power Pop,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,054,Actions - South Of The Water,Power Pop,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,055,Angels In Amplifiers - I m Alright,Smooth Acoustic Rock,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.525..."


## VI - SAUVEGARDE :

In [15]:
#!pip install parquet
#df_multi.head(10).to_parquet('df_multi')
#df_vocals.to_parquet('df_vocals')


In [16]:
np.save('df_multi', df_multi.head(10))
np.save('df_vocals',df_vocals)